In [11]:
import glob
import pandas as pd
import json
import datetime
from tqdm import tqdm
import os
from django.conf import settings
#from sqlalchemy import create_engine
os.environ["DJANGO_SETTINGS_MODULE"] = "mysite.settings"
import django
django.setup()
from tqdm import tqdm
import glob
import numpy as np
import numpy as np

from sklearn.neighbors import KDTree
comfig_file_path = '../../main_config.txt'
main_name = 'yandex/'
sub_name = 'raw_org_search/'

In [12]:
import sys
import json
sys.path.append('/usr/lib/python3/dist-packages')
from main_map.models import Organization, OrganizationNatClass, Workplace, House, MetroStation, Metro, GroundStop

In [13]:
from pyproj import Proj, transform

PROJ3857 = Proj("+init=EPSG:3857")
PROJ4326 = Proj("+init=EPSG:4326")

In [14]:
nat_class = ['Магазин бытовой техники и электроники']

In [15]:
xy = ['x', 'y']

In [16]:
filter_geo = {}

In [17]:
lat, lon = 55.855500, 37.441124
x, y = transform(PROJ4326, PROJ3857, lon, lat)
def get_cann_dict(x,y):

    half_side = 5e4
    influence_mult = 2
    our_brands = ['М.Видео','Эльдорадо']
    filter_dic = {
                'x__gt' : x - half_side,
                'y__gt' : y - half_side,
                'x__lt' : x + half_side,
                'y__lt' : y + half_side,
                'nat_class' : 'Магазин бытовой техники и электроники',
    }

    objects = OrganizationNatClass.objects.filter(**filter_dic).values()
    df_orgs = pd.DataFrame(list(objects))
    df_orgs['R'] = ((df_orgs['x'] - x)**2 + (df_orgs['y'] - y)**2)**0.5

    df_we_sorted = df_orgs[df_orgs['chain_name'].isin(our_brands)].sort_values('R').iloc[:3]
    dics = []
    for i, row in df_we_sorted.iterrows():
        dic = {}
        dic['R'] = row['R']
        dic['address'] = row['address']
        dic['chain_name'] = row['chain_name']
        R_boarder = row['R']

        n_comp = (df_orgs['R'].min()/df_orgs[df_orgs['R']<=(influence_mult*R_boarder)]['R']   ).sum()

        dic['cannib'] = n_comp/(1+n_comp)
        dics.append(dic)
    return dics

In [42]:
dics = get_cann_dict(x,y)
cann_formatted = ''
for dic in dics:
    cann_formatted += dic['chain_name'] + '</br>'
    cann_formatted += 'Адрес: ' + dic['address'] + '</br>'
    cann_formatted += 'Расстояние до новой точки: ' + str(int(dic['R'])) + '</br>'
    cann_formatted += 'Каннибализм: ' + str(int(100*(dic['cannib']-1))) + '%</br></br>'

In [41]:
cann_formatted

'М.Видео</br>Адрес: Сходненская ул., 56, Москва, Россия</br>Расстояние до новой точки: 1153</br>Каннибализм: 63</br></br>Эльдорадо</br>Адрес: Ленинградское ш., 112/1, Москва, Россия</br>Расстояние до новой точки: 3157</br>Каннибализм: 79</br></br>М.Видео</br>Адрес: Фестивальная ул., 2Б, Москва, Россия</br>Расстояние до новой точки: 4179</br>Каннибализм: 82</br></br>'

In [37]:
cann_formatted

'М.Видео</br>Адрес: Сходненская ул., 56, Москва, РоссияАдрес: Сходненская ул., 56, Москва, Россия</br>Адрес: Сходненская ул., 56, Москва, Россия</br>Адрес: Сходненская ул., 56, Москва, Россия</br>Расстояние до новой точки: 1153</br>Каннибализм: 63'